<h1 align="center">
  <a href="https://uptrain.ai">
    <img width="300" src="https://user-images.githubusercontent.com/108270398/214240695-4f958b76-c993-4ddd-8de6-8668f4d0da84.png" alt="uptrain">
  </a>
</h1>

<h1 style="text-align: center;">Collecting Finetuning Dataset for Conversation Summarization</h1>

**Objective**: Collect a finetuning dataset to improve a model that summarizes human conversations.

**Model**: We are working on the `facebook/bart-large-xsum model` that was finetuned on the SAMSum dataset (available [here](https://huggingface.co/lidiya/bart-large-xsum-samsum)).
It is one of the [top performers](https://paperswithcode.com/sota/abstractive-text-summarization-on-samsum) in open-source models on the SAMSum corpus.

**Dataset**: Our model has been finetuned on the [SAMSum corpus](https://huggingface.co/datasets/samsum) which has 16k conversations and their summaries. Additionally, we evaluate our model on the [DialogSum corpus](https://huggingface.co/datasets/knkarthick/dialogsum) which has 13k conversations and their summaries. We note that the model has good performance on this new dataset as well, but slightly worse than its performance on the SAMSum dataset. Our objective is to create a finetuning dataset to improve the model on DialogSum like conversations.

**Method**: We employ several techniques to collect the fine-tuning dataset: 
1) Edge-case Collection (user defines the edge-case parameters) 
2) Building Custom Monitor (that checks out-of-vocabulary cases) 
3) Finding points where data drift has occured 
4) Finding clusters around points where accuracy is low
5) Visualizing UMAP/t-SNE for low-performing clusters

#### Install required packages

In [1]:
# !pip install uptrain rouge-score datasets umap-learn matplotlib py7zr 

In [2]:
from datasets import load_dataset
import json
import numpy as np
import os
import uptrain
from rouge import Rouge 
import random
from matplotlib import pyplot as plt
import pandas as pd
import subprocess
import zipfile

/Users/vipul/opt/anaconda3/envs/ut_test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'rouge'

#### Load Datasets from Hugging Face

In [ ]:
samsum_dataset = load_dataset("samsum")
dialogsum_dataset = load_dataset("knkarthick/dialogsum")

#### Download model outputs and their embeddings from remote
We understand that running the bart-large-xsum can be time consuming on sum machines, hence, we have pre-generated the model outputs and their corresponding sentence BERT embeddings to remote for both the SAMSum and DialogSUM datasets. Due to this, running this entire script does not take too much time (e.g., it runs in 3 minutes on my Macbook Air).

In [ ]:
remote_url = "https://oodles-dev-training-data.s3.amazonaws.com/conversation_summarization_data.zip"
data_dir = 'data'
if not os.path.exists(data_dir):
    file_downloaded_ok = subprocess.call("wget " + remote_url, shell=True, 
                                         stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    print("Data downloaded.")
with zipfile.ZipFile('conversation_summarization_data.zip', 'r') as zip_ref:
    zip_ref.extractall("./")
print("Prepared Model Outputs.")
os.remove('conversation_summarization_data.zip')

Next, we analyze the above model outputs and define edge cases as follows.

### Edge-case type 1

**Observation**: Model performs badly for long dialogs. For example, it generates the following (incomplete) summaries for long dialogs:

```
"Benjamin, Elliot, Daniel and Hilary are going to have lunch with French"
"Jesse, Lee, Melvin and Maxine are going to chip in for the"
"Jayden doesn't want to have children now, but maybe in the future when"
"Leah met a creepy guy at the poetry reading last night. He asked her"
"Jen wants to break up with her boyfriend. He hasn't paid her back the"
```

Next, we generate a histogram of length of input dialogues on the training dataset (i.e., SAMSum train). From here, we note that a length of 1700 can be a good cut-off to collect large conversation data-points.

In [ ]:
a = [len(x) for x in samsum_dataset['train']['dialogue']]
fig, ax = plt.subplots(figsize =(7, 3))
ax.hist(a, bins=16)
ax.set_xlabel('Input dialog length')
ax.set_ylabel('Number of data points')
plt.show()

In [ ]:
"""
Check if the length of the input is greater 
than 1700 characters.
"""
def length_check_func(inputs, outputs, gts=None, extra_args={}):
    this_batch_dialog = inputs['dialog']
    return np.array([len(x) for x in this_batch_dialog]) > 1700

edge_case_length = {
    'type': uptrain.Monitor.EDGE_CASE,
    'signal_formulae': uptrain.Signal("Length_dialog", length_check_func)
} 

### Edge-case type 2

**Observation**: When the model is not able to summarize well, it just copies one or two sentences. This may work in general but performs very badly when we have a negation in the conversation. See the following examples: <br>

```
Input: 
Janice: my son has been asking me to get him a hamster for his birthday. Janice: Should I? Martina: NO! NO! NO! NO! NO! Martina: I got one for my son and it stank up the whole house. Martina: So don't do it!!!
Output: Janice's son wants her to get him a hamster for his birthday.

Input: 
Person1: Hello, I'm looking for a shop that sells inexpensive cashmere sweaters. Person2: Have you tried an outlet?Person1: Why didn't I think of that? Person2: Many of my friends shop at outlets. Person1: Thanks. That is a good suggestion. Person2: I'm only too happy to help.
Output: Person1 is looking for a shop that sells inexpensive cashmere sweaters.
```

In [ ]:
# Checking whether sentences from inputs are copied directly using Rogue-L metric
def rogueL_check_func(inputs, outputs, gts=None, extra_args={}):
    r = Rouge()
    res = r.get_scores([x.lower() for x in inputs['dialog']],[x.lower() for x in outputs])
    rogue_l = [x['rouge-l']['f'] for x in res]
    return np.array(rogue_l)

# Cheking whether there's a negation in the input
def negation_func(inputs, outputs, gts=None, extra_args={}):
    has_negation = []
    for text in inputs['dialog']:
        this_has_negation = False
        all_words = text.split()
        for negation_word in ['no', 'not', "can't", "couldn't", "won't", "didn't", "don't"]:
            if negation_word in all_words:
                this_has_negation = True
        has_negation.append(this_has_negation)
    return has_negation

edge_case_negation = {
    'type': uptrain.Monitor.EDGE_CASE,
    'signal_formulae': (uptrain.Signal("Rogue-L", rogueL_check_func) > 0.3) 
        & uptrain.Signal("Has_negation", negation_func)
}

#### Custom Monitor to check Vocabulary Coverage

In this case, we define a custom monitor to see what's the average vocabulary coverage of the new dataset (i.e., DialogSum) on the old dataset (i.e., SAMSum). Defining a custom metric to check if there is a shift in vocabulary. Note that unlike previous edge cases checks that were stateless, this is a stateful check that contains the training vocabulary information.

In [ ]:
from collections import Counter

def clean_string(x):
    x = x.lower()
    x = x.replace('.', '')
    x = x.replace(',', '')
    x = x.replace('\'', '')
    x = x.replace('?', '')
    x = x.replace('#', '')     
    x = x.replace(':', '')  
    x = x.replace('!', '')  
    return x

# Define the training vocabulary
all_text = ""
for x in samsum_dataset['train']['dialogue']:
    all_text += clean_string(x) + " "  
vocab = Counter(all_text.split())

"""
Used to define a state which contains the training set 
vocabulary and the out-of-vocab words (and their count).
"""
def vocab_init(self):
    # Reference (i.e. training) vocabulary
    self.vocab = set(vocab.keys())   
    self.vocab_arr = []
    self.out_of_vocab_words = Counter()

"""
This is the actual check that checks the vocabulary coverage 
of the production dataset in the training dataset.
"""
def vocab_drift(self, inputs, outputs, gts=None, extra_args={}):
    for x in inputs['dialog']:
        x_s = set(clean_string(x).split())
        self.vocab_arr.append(len(x_s & self.vocab)/len(x_s))
        outside_words = x_s - self.vocab
        self.out_of_vocab_words.update(Counter(outside_words))
        
        # Save 50 most common out of vocabulary words
        with open("out_of_vocab_words.json", "w") as f:
            json.dump(self.out_of_vocab_words.most_common(50), f)
        
        # Calculate vocabulary coverage
        count = len(self.vocab_arr)
        coverage = 100*sum(self.vocab_arr)/count

        # Logging to UpTrain dashboard
        self.log_handler.add_scalars('vocab coverage', 
                {'y_coverage': coverage},
            count, 'vocab_coverage', file_name='vocab_coverage')

# Defining a custom monitor check for vocabulary coverage
custom_monitor_check = {
    'type': uptrain.Monitor.CUSTOM_MONITOR,
    'initialize_func': vocab_init,
    'check_func': vocab_drift,
    'need_gt': False,
}

#### Defining UpTrain Config and Framework

In [ ]:
config = {
    "checks": [edge_case_negation, edge_case_length, custom_monitor_check],
    "logging_args": {"st_logging": True},
    "retraining_folder": "smart_data_edge_case_and_custom_monitor",
}

framework = uptrain.Framework(cfg_dict=config)

### Running model in production and logging data to UpTrain

In [ ]:
# Run the model in production
def run_production(framework, batch_size=200):
    for dataset_name in ['samsum', 'dialogsum']:
        if dataset_name=='samsum':
            d_type = 'test'
            dataset = samsum_dataset[d_type]
        elif dataset_name=='dialogsum':
            d_type = 'train'
            dataset = dialogsum_dataset[d_type]
        else:
            raise Exception("Dataset Error")

        f = open(os.path.join(data_dir, f"out_{d_type}_{dataset_name}_summaries.json"))
        all_summaries = json.load(f)
        f.close()

        """
        Note: We use sentence BERT embeddings generated from here:
        https://huggingface.co/sentence-transformers
        But any other embeddings, such as the ones generated by the
        encoder can be used as well.
        """
        f = open(os.path.join(data_dir, f"out_{d_type}_{dataset_name}_bert_embs.json"))
        all_bert_embs = json.load(f)
        f.close()

        for idx in range(len(all_bert_embs)//batch_size):
            idxs = slice(idx*batch_size, (idx+1)*batch_size)
            this_batch = dataset['summary'][idxs]
            this_batch_dialog = dataset['dialogue'][idxs]

            inputs = {
                'id': list(range(idx*batch_size, (idx+1)*batch_size)),
                'bert_embs': np.array(all_bert_embs[idxs]),
                'dataset_label': [dataset_name]*batch_size,
                'dialog': this_batch_dialog,
                'summary': this_batch,
            }
            idens = framework.log(inputs=inputs, outputs=all_summaries[idxs])
        print(f"{(idx+1)*batch_size} predictions logged for {dataset_name} {d_type}")

In [ ]:
run_production(framework)

#### Vocabulary coverage

We obtain the following plot from the Custom monitor to check vocabulary coverage in production data. Initially, (for SAMSum test), the coverage is ~98%, but later (for DialogSum), the coverage decreases to ~95%.

<img width="550" alt="concept_drift_avg_acc" src="https://uptrain-demo.s3.us-west-1.amazonaws.com/conversation_summarization/vocab_coverage.gif">

#### Checking the collected edge cases

In [ ]:
# Print edge-cases collected for each reason
def print_edge_cases(csv_file, num_per_reason=2):
    df = pd.read_csv(csv_file)
    reasons_covered = Counter()
    
    for idx in range(len(df)):
        reason = [df['reasons'][idx]]
        count = reasons_covered.get(reason[0], 0)
        if count >= num_per_reason:
            continue
        reasons_covered.update(reason)
        print('Reason:           ', reason[0])
        print('Output:           ', df['output'][idx]) 
        print('Annotated Summary:', df['summary'][idx]) 
        print('Dialogue:         ', df['dialog'][idx])
        print('')

In [ ]:
print_edge_cases(config['retraining_folder'] + "/1/smart_data.csv")

#### Get out-of-vocab-words 

In [ ]:
f = open("out_of_vocab_words.json")
out_of_vocab_words = json.load(f)
f.close()
out_of_vocab_words = [x[0] for x in out_of_vocab_words]
print(out_of_vocab_words)

Note from the above how a lot of the words are related to Asia (such as yuan, li, wang, taiwan, zhang, liu, chinas, sichaun, singapore, etc.). This implies that a lot of converation in the Dialogsum datasets are focused on the Asia region. Next, we define a edge-case check to catch these cases.

### Applying a check for Asian words on production data

In [ ]:
asian_words = ['yuan', 'li', 'wang', 'taiwan', 'zhang', 'liu', 'chinas', 'sichaun', 'singapore']
def asian_words_check(inputs, outputs, gts=None, extra_args={}):
    has_asian_word = [False]*len(inputs['dialog'])
    for i,text in enumerate(inputs['dialog']):
        all_words = clean_string(text).split()
        if len(set(asian_words).intersection(set(all_words))):
            has_asian_word[i] = True
    return has_asian_word

edge_case_asian_word = {
    'type': uptrain.Monitor.EDGE_CASE,
    'signal_formulae': uptrain.Signal("asian_word", asian_words_check)
}

### Data Drift Detection

Finding points where BERT embeddings are further than the BERT emneddings of the reference dataset (SAMSum training in this case).

#### Define helper functions

In [ ]:
"""
Using training data (i.e., SAMSum train), we generate and save a reference 
dataset to be used by the UpTrain framework. This dataset is used to detect 
drift, apply dimensioanlity reductions and compare visualizations.
"""
def generate_reference_dataset(summary, output_summaries_file, bert_embs_file, file_name, dataset_label):
    data = []
    if not os.path.exists(file_name):
        
        # Load model output summaries 
        f = open(output_summaries_file)
        output_summaries = json.load(f)
        f.close()
        
        # Load respective BERT embeddings of output summaries
        f = open(bert_embs_file)
        bert_embs = list(json.load(f))
        f.close()
        
        data = []
        for idx in range(len(bert_embs)):
            if isinstance(dataset_label, str):
                data.append({
                    'id': idx,
                    'dataset_label': dataset_label,
                    'summary': summary[idx],
                    'bert_embs': list(bert_embs[idx]),
                    'output': output_summaries[idx],
                })

        with open(file_name, "w") as f:
            json.dump(data, f, cls=uptrain.UpTrainEncoder)
        print("Generated reference dataset.")
    else:
        print("Reference dataset exists. Skipping generating again.")

#### Generate reference dataset to calculate drift and dimensionality reduction

In [ ]:
data_dir = 'data'

# Get the locations of training-related data and outputs
output_summaries_file = os.path.join(data_dir, 'out_train_samsum_summaries.json')
bert_embs_file = os.path.join(data_dir, 'out_train_samsum_bert_embs.json')
reference_dataset_file = os.path.join(data_dir, 'reference_dataset.json')

# Generate and save reference dataset if not available 
generate_reference_dataset(samsum_dataset['train']['summary'], output_summaries_file, 
                           bert_embs_file, reference_dataset_file, 'reference')

#### Define a performance metric

In [ ]:
"""
Define a performance metric. We use Rogue-L
similarity, but choose any metric that is 
relevant to your use-case.
"""
def rogue_l_similarity(text1_list, text2_list):
    r = Rouge()
    res = r.get_scores([x.lower() for x in text1_list],[x.lower() for x in text2_list])
    return [x['rouge-l']['f'] for x in res]

#### Get Rogue-L Performance scores on the Dialogsum Data 

In [ ]:
file = os.path.join(data_dir, "out_train_dialogsum_summaries.json")
with open(file) as f:
    dialogsum_summaries = json.load(f)

dialogsum_gts = dialogsum_dataset['train']['summary'][0:len(dialogsum_summaries)]
dialogsum_scores = rogue_l_similarity(dialogsum_summaries, dialogsum_gts)

dialogsum_train_bert_embs_file = os.path.join(data_dir, 'out_train_dialogsum_bert_embs.json')
with open(dialogsum_train_bert_embs_file) as f:
    dialogsum_train_bert_embs = np.array(json.load(f))

#### Select bad-performing data-points

In [ ]:
# Select data-points where Rogue-L scores are 0.0
outlier_idxs = np.where(np.array(dialogsum_scores) <= 0.0)[0]
selected_outliers = dialogsum_train_bert_embs[outlier_idxs, :]

#### Defining a Data-drift Check

In [ ]:
drift_check = {
    'type': uptrain.Monitor.DATA_DRIFT,
    'is_embedding': True,
    'measurable_args': {
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'bert_embs'
    },
    'reference_dataset': reference_dataset_file,
    # Number of clusters to calculate data drift
    "num_buckets": 50,
    # Number of points to wait before calculating drift
    "initial_skip": 500,
    # Outliers around which we want to collect data-points
    'outlier_data': selected_outliers,
    # Threshold for Earth-moving-distance (EMD) to collect drift points
    'emd_threshold': 10
}

#### Defining Dimensionality Reduction and Visualization using UMAP

In [ ]:
umap_check = {
    'type': uptrain.Visual.UMAP,
    "measurable_args": {
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'bert_embs'
    },
    "label_args": {
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'dataset_label'
    },
    "hover_args": [
    {
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'id'
    },
    {
        'type': uptrain.MeasurableType.PREDICTION,
        'feature_name': 'output'
    },
    {
        'type': uptrain.MeasurableType.INPUT_FEATURE,
        'feature_name': 'summary'
    },
    ],
    "update_freq": 13200,
    'initial_dataset': reference_dataset_file,
    "do_clustering": False,
}

#### Again, we define the UpTrain Config and Framework

In [ ]:
config = {
    "checks": [edge_case_asian_word, drift_check, umap_check],
    "logging_args": {"st_logging": True},
    "retraining_folder": "data_drift_smart_data",
}

framework = uptrain.Framework(cfg_dict=config)

In [ ]:
run_production(framework)

#### Comparing performance over collected points that are close to the outliers (Disclaimer: It's 0.7 less)

In [ ]:
print("Overall Accuracy (Rogue-L): ", np.mean(dialogsum_scores))

In [ ]:
smart_data = pd.read_csv(config['retraining_folder'] + '/1/smart_data.csv') 
smart_data = smart_data[smart_data['reasons']=='"Close_to_User_annotated_Outliers"'].to_dict('records')
smart_data_scores = rogue_l_similarity([eval(x['output']) for x in smart_data], 
                                            [eval(x['summary']) for x in smart_data])
print("Accuracy on clusters around user-picked outliers", np.mean(smart_data_scores))

Note how the accuracy obtained on points that are closer to outliers is worse compared to the overall accuracy.

#### Checking the collected edge cases

In [ ]:
print_edge_cases(config['retraining_folder'] + "/1/smart_data.csv")

#### UMAP Visualization

Datasets marked `reference` (i.e., SAMSum training) and `samsum` (i.e., SAMSum test) are close in the UMAP space. Most point from the DialogSum dataset are further than the data on which the model was finetuned on (i.e., reference aka SAMSum train).  

![](https://uptrain-demo.s3.us-west-1.amazonaws.com/conversation_summarization/umap_conv_summ.png)